# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

### Skąd wziąć dużą liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**

Test probabilistyczny, np. Rabina-Millera. 

In [1]:
import random

def isPrime(n, k): # n - testowana liczba, k - liczba prób
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False    # speedup

    s = 0
    d = n-1
    while d % 2 == 0:
        s += 1
        d //= 2 # n = 2^s * d + 1

    #tutaj napisz swój kod 
    for i in range(k):
        a = random.randint(2, n-1)
        x = pow(a, d) % n
        if (x != 1) and (x != n-1):
            for j in range(1, s):
                if (x == n-1): break; # wykryliśmy liczbę silnie pierwszą spróbujmy innego świadka
                x = pow(x, 2) % n
            if (x != n-1): return False # żaden wyraz nie przystawał -1, więc to jest liczba na pewno złożona

    return True

for i in range(65500,65600):            
    if(isPrime(i,10)): print(i)

65519
65521
65537
65539
65543
65551
65557
65563
65579
65581
65587
65599


### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [2]:
def fastModularExponentiation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m

### Obliczanie elementu odwrotnego w ciele skończonym
1. Algorytm Euklidesa - znajdowanie największego wspólnego dzielnika
2. Rozszerzony Algorytm Euklidesa - obliczanie elementu odwrotnego względem mnożenia w ciele skończonym

In [3]:

## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

# wersja rekurencyjna
def gcdExtendedRecursion(a, b):  
    # przypadek trywialny
    if a == 0 :   
        return b,1,0             
    gcd,x1,y1 = gcdExtendedRecursion(b%a, a) #rekurencyjny Euklides      
   
    # sumowanie powrotne  
    y = x1 - (b//a) * y1  
    x = y1  
    
    return gcd,x,y 

#wersja iteracyjna
def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m


a, b = 154,801
g, x, y = gcdExtendedRecursion(a, b)  
print("gcd(", a , "," , b, ") = ", g, x, y)
print("Element odwrotny do ", a, "modulo", b, ":", y%b)

print(findModInverse(154,801))

gcd( 154 , 801 ) =  1 5 -26
Element odwrotny do  154 modulo 801 : 775
775


## Generowanie kluczy w kryptosystemie RSA
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu.
### Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [4]:
#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize):
    #napisz swój kod tutaj
    num = 0
    while not isPrime(num, 10):
        num = random.getrandbits(keysize)
    return num

print(generatePrime(16))

58151


## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [5]:
import random, sys, os

def generateKey(keySize):
    #napisz swój kod tutaj
    p = generatePrime(keySize)
    q = generatePrime(keySize)
    
    n = p * q
    
    e = 65537
    
    d = findModInverse(e, (p - 1) * (q - 1))
    
    publicKey = (n, e)
    privateKey = (n, d)
    
    return (publicKey, privateKey)

pub,priv = generateKey(16)
print(pub,priv)

(1324007507, 65537) (1324007507, 560930369)


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [6]:
def encrypt(message, modulus, exp):
    # kod szyfrowania
    ascii_message = list(map(lambda x: ord(x), message))
  
    message_encrypted = list(map(lambda x: pow(x, exp, modulus), ascii_message))
    
    return message_encrypted


ciphertext = encrypt('Hello world!', pub[0], pub[1])
print(ciphertext)

[26212566, 9693506, 589462100, 589462100, 629186665, 1186005213, 1040697257, 629186665, 423659384, 589462100, 545464356, 100539840]


### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [7]:
def decrypt(message_encrypted, modulus, exp):
    ascii_message = list(map(lambda x: pow(x, exp, modulus), message_encrypted))
  
    message = list(map(lambda x: chr(x), ascii_message))
  
    return ('').join(message)

message = decrypt(ciphertext, priv[0], priv[1])
print(message)

Hello world!


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie (dla chętnych)
1. Zademonstruj deformowalność podręcznikowego szyfrowania RSA
2. Zaimplementuj dopełnienie OAEP (_Optimal Assymmetric Encryption Padding_)


In [20]:
# Deformowalność
# w podręcznikowej implementacji RSA można bez konsekwenjci zamieniać miejscami bloki wiadomości
# bez wykrycia przez komunikujących się i bez deszyfrowania

import copy

# szyfrogram oryginału
cip_og = encrypt('Moja ulubiona liczba to - 123', pub[0], pub[1])

# zdeformowany szyfrogram
cip_tamp = copy.deepcopy(cip_og)
cip_tamp[-3], cip_tamp[-1] = cip_tamp[-1], cip_tamp[-3]
cip_tamp[-2], cip_tamp[-1] = cip_tamp[-1], cip_tamp[-2]

# zdeszyfrowany oryginał
mes_og = decrypt(cip_og, priv[0], priv[1])
print("Oryginalna wiadomość:", mes_og)

# zdeszyfrowany zdeformowany szyfrogram
mes_tamp = decrypt(cip_tamp, priv[0], priv[1])
print("Zdeformowana:", mes_tamp)

Oryginalna wiadomość: Moja ulubiona liczba to - 123
Zdeformowana: Moja ulubiona liczba to - 312
